# 00_Base_P

In [1]:
import numpy as np
import random
import torch
torch.cuda.set_device(2)

In [2]:
random_seed = 617
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)
print("Random Seed:",random_seed)

Random Seed: 617


In [3]:
# Import utils from a specific defense
from defenses.base.loader import get_loader
from defenses.base.model import get_model
from defenses.base.train import Trainer, get_acc

Loader : Base
Model : Base
Train : Base


In [4]:
# Set Name
NAME = "00_Base_P"

## 1. Train

In [5]:
# Get Train, Test datasets
train_loader_ns, _ = get_loader(shuffle_train=False)
train_loader, test_loader = get_loader(shuffle_train=True)

# Get Model
model = get_model(name="PreAct18").cuda()

Files already downloaded and verified
Files already downloaded and verified
Data Loaded!
Train Data Length : 50000
Test Data Length : 10000
Files already downloaded and verified
Files already downloaded and verified
Data Loaded!
Train Data Length : 50000
Test Data Length : 10000
WRN-28 is loaded.


In [6]:
train_sets = iter(train_loader_ns).next()
test_sets = iter(test_loader).next()

In [7]:
# Train Model
trainer = Trainer(model, train_sets, test_sets)
trainer.train(train_loader, epochs=30,
              save_path="./_models/"+NAME, scheduler="Cyclic", 
              record_type="Epoch", save_type="Epoch")

Train Information:
-Epochs: 30
-Optimizer: SGD (
Parameter Group 0
    dampening: 0
    initial_lr: 0
    lr: 0.0
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0005
)
-Scheduler: <torch.optim.lr_scheduler.CyclicLR object at 0x7f47beef8b70>
-Save Path: ./_models/00_Base
-Save Type: Per Epoch
-Record Type: Per Epoch
-Device: cuda


KeyboardInterrupt: 

In [ ]:
save_path = "./_models/"+NAME
torch.save(trainer.model.cpu().state_dict(), save_path+".pth")
trainer.rm.to_csv(save_path+".csv")    

In [ ]:
print("Final Test Acc : %2.2f"%get_acc(model, test_loader))

## 2. Record Visualization

In [ ]:
# Plot Train and Test Accuracy
rm.plot(title=NAME, xlabel="Epoch", ylabel="Accuracy", ylabel_second="Loss",
        x_key='Epoch', y_keys=['Acc', 'Clean(Te)', 'FGSM(Te)', 'PGD(Te)', 'GN(Te)'],
        ylim=(-10, 110), colors=['k', 'r', 'g', 'b', 'c'])

## 3. White Box Attack

In [ ]:
import torchattacks
from torchattacks import FGSM, PGD, MultiAttack

In [ ]:
model.load_state_dict(torch.load("./_models/"+NAME+".pth"))
model = model.cuda().eval()

### 3.1 FGSM

In [ ]:
fgsm = FGSM(model, eps=8/255)
fgsm.set_mode('int')
fgsm.save(data_loader=test_loader,
          file_name="./_datasets/"+NAME+"_FGSM.pt", accuracy=True)

### 3.2 PGD

In [ ]:
# WB2 : PGD
pgd = PGD(model, alpha=2/255, eps=8/255, iters=50)
pgd.set_mode('int')
pgd.save(data_loader=test_loader,
         file_name="./_datasets/"+NAME+"_PGD.pt", accuracy=True)

### 3.3 PGD with 10 Random Restarts

In [ ]:
# WB3 : PGD
pgd = PGD(model, alpha=2/255, eps=8/255, iters=50, random_start=True)
multi = MultiAttack(model, [pgd]*10)
multi.set_mode('int')
multi.save(data_loader=test_loader,
           file_name="./_datasets/"+NAME+"_RPGD.pt", accuracy=True)

## 4. Black Box Attack

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# BB1 : WRN_28_20
adv_images, adv_labels = torch.load("./_datasets/00_WRN_28_20_PGD.pt")
adv_data = TensorDataset(adv_images.float()/255, adv_labels)
adv_loader = DataLoader(adv_data, batch_size=128, shuffle=False)
get_acc(model, adv_loader)

In [ ]:
# BB2 : WRN_40_10
adv_images, adv_labels = torch.load("./_datasets/00_WRN_40_10_PGD.pt")
adv_data = torch.utils.data.TensorDataset(adv_images.float()/255, adv_labels)
adv_loader = DataLoader(adv_data, batch_size=128, shuffle=False)
get_acc(model, adv_loader)